In [23]:
import tensorflow as tf

# 하이퍼파라미터 설정

In [24]:
# 하이퍼파라미터
EPOCHS = 10
NUM_WORDS = 10000

# 모델 정의

In [25]:
# 모델 정의
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
    self.rnn = tf.keras.layers.SimpleRNN(32)
    self.dense = tf.keras.layers.Dense(2, activation='softmax')

  def call(self, x, training=None, mask=None):
    x = self.emb(x)
    x = self.rnn(x)
    return self.dense(x)

# 학습, 테스트 루프 정의

In [26]:
### 학습 루프
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)

  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

### 테스트
@tf.function
def test_step(model, seqs, labels, loss_object, test_loss, test_accuracy):
  predictions = model(seqs, training=False)

  t_loss = loss_object(labels, predictions)
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [27]:
# 데이터셋 로드
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 학습 환경 정의
모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [28]:
# 모델 생성
model = MyModel()

# 손실함수, 최적화 알고리즘 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 평가지표 정의
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 학습 루프 동작

In [29]:
for epoch in range(EPOCHS):
  for seqs, labels in train_ds:
    train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)
  
  for test_seqs, test_labels in test_ds:
    test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.5344611406326294, Accuracy: 71.85599517822266, Test Loss: 0.45841047167778015, Test Accuracy: 78.5040054321289
Epoch 2, Loss: 0.369983047246933, Accuracy: 83.80000305175781, Test Loss: 0.4671441614627838, Test Accuracy: 78.30799865722656
Epoch 3, Loss: 0.23231709003448486, Accuracy: 90.81599426269531, Test Loss: 0.568793773651123, Test Accuracy: 76.02799987792969
Epoch 4, Loss: 0.10209940373897552, Accuracy: 96.4280014038086, Test Loss: 0.8056340217590332, Test Accuracy: 75.38800048828125
Epoch 5, Loss: 0.041172340512275696, Accuracy: 98.71199798583984, Test Loss: 1.0237432718276978, Test Accuracy: 75.50399780273438
Epoch 6, Loss: 0.018897566944360733, Accuracy: 99.45999908447266, Test Loss: 1.1751729249954224, Test Accuracy: 75.56399536132812
Epoch 7, Loss: 0.015160083770751953, Accuracy: 99.52799987792969, Test Loss: 1.2331442832946777, Test Accuracy: 74.93199920654297
Epoch 8, Loss: 0.019736450165510178, Accuracy: 99.3479995727539, Test Loss: 1.3276941776275635, Tes